In [366]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from sklearn.metrics import classification_report
from sklearn import svm

In [295]:
df=pd.read_csv("classification.csv")

In [296]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [297]:
newdf=df.sample(n=60000)

In [298]:
newdf.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
3268477,251,CASH_OUT,77799.25,C1236314568,0.00,0.00,C1165834029,845019.90,922819.15,0,0
1580851,155,PAYMENT,9258.43,C1334956157,0.00,0.00,M1275633310,0.00,0.00,0,0
5746645,399,CASH_IN,42402.44,C2077763870,15691627.24,15734029.68,C174011890,626570.64,584168.19,0,0
561392,22,CASH_OUT,22201.76,C2014200800,31748.82,9547.06,C867882420,1553217.73,1575419.49,0,0
768127,39,CASH_OUT,129617.14,C973219994,16571.00,0.00,C1273191747,413656.24,543273.38,0,0


In [299]:
newdf['isFlaggedFraud'].value_counts()

0    60000
Name: isFlaggedFraud, dtype: int64

In [300]:
newdf.isnull().values.any()

False

In [301]:
newdf['isFraud'].value_counts()

0    59946
1       54
Name: isFraud, dtype: int64

In [302]:
df['isFlaggedFraud'].nunique()

2

In [303]:
newdf.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [304]:
print("Number of unique values of type: "+str(newdf['type'].nunique()))
print("Number of unique values of nameOrig: "+str(newdf['nameOrig'].nunique()))
print("Number of unique values of nameDest: "+str(newdf['nameDest'].nunique()))

Number of unique values of type: 5
Number of unique values of nameOrig: 60000
Number of unique values of nameDest: 57392


In [305]:
newdf.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
3268477,251,CASH_OUT,77799.25,C1236314568,0.00,0.00,C1165834029,845019.90,922819.15,0,0
1580851,155,PAYMENT,9258.43,C1334956157,0.00,0.00,M1275633310,0.00,0.00,0,0
5746645,399,CASH_IN,42402.44,C2077763870,15691627.24,15734029.68,C174011890,626570.64,584168.19,0,0
561392,22,CASH_OUT,22201.76,C2014200800,31748.82,9547.06,C867882420,1553217.73,1575419.49,0,0
768127,39,CASH_OUT,129617.14,C973219994,16571.00,0.00,C1273191747,413656.24,543273.38,0,0


In [306]:
newdf = pd.get_dummies(newdf,prefix=['type'], columns = ['type'])
newdf.head()


,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
3268477,251,77799.25,C1236314568,0.00,0.00,C1165834029,845019.90,922819.15,0,0,0,1,0,0,0
1580851,155,9258.43,C1334956157,0.00,0.00,M1275633310,0.00,0.00,0,0,0,0,0,1,0
5746645,399,42402.44,C2077763870,15691627.24,15734029.68,C174011890,626570.64,584168.19,0,0,1,0,0,0,0
561392,22,22201.76,C2014200800,31748.82,9547.06,C867882420,1553217.73,1575419.49,0,0,0,1,0,0,0
768127,39,129617.14,C973219994,16571.00,0.00,C1273191747,413656.24,543273.38,0,0,0,1,0,0,0


In [320]:
newdf=newdf.reset_index()

In [321]:
newdf.head()

,index,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,3268477,251,77799.25,C1236314568,0.00,0.00,C1165834029,845019.90,922819.15,0,0,0,1,0,0,0
1,1580851,155,9258.43,C1334956157,0.00,0.00,M1275633310,0.00,0.00,0,0,0,0,0,1,0
2,5746645,399,42402.44,C2077763870,15691627.24,15734029.68,C174011890,626570.64,584168.19,0,0,1,0,0,0,0
3,561392,22,22201.76,C2014200800,31748.82,9547.06,C867882420,1553217.73,1575419.49,0,0,0,1,0,0,0
4,768127,39,129617.14,C973219994,16571.00,0.00,C1273191747,413656.24,543273.38,0,0,0,1,0,0,0


In [322]:
newdf['nameDest'][0]

'C1165834029'

In [323]:
i=0
while i<len(newdf['nameDest']):
    newdf['nameDest'][i]=newdf['nameDest'][i][0]
    i=i+1

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [324]:
newdf.head()
#x.nunique()

,index,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,3268477,251,77799.25,C1236314568,0.00,0.00,C,845019.90,922819.15,0,0,0,1,0,0,0
1,1580851,155,9258.43,C1334956157,0.00,0.00,M,0.00,0.00,0,0,0,0,0,1,0
2,5746645,399,42402.44,C2077763870,15691627.24,15734029.68,C,626570.64,584168.19,0,0,1,0,0,0,0
3,561392,22,22201.76,C2014200800,31748.82,9547.06,C,1553217.73,1575419.49,0,0,0,1,0,0,0
4,768127,39,129617.14,C973219994,16571.00,0.00,C,413656.24,543273.38,0,0,0,1,0,0,0


In [326]:
newdf = pd.get_dummies(newdf,prefix=['nameDest'], columns = ['nameDest'])
newdf.head()
#simplifieddf['isFlaggedFraud'].nunique()

,index,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,nameDest_C,nameDest_M
0,3268477,251,77799.25,C1236314568,0.00,0.00,845019.90,922819.15,0,0,0,1,0,0,0,1,0
1,1580851,155,9258.43,C1334956157,0.00,0.00,0.00,0.00,0,0,0,0,0,1,0,0,1
2,5746645,399,42402.44,C2077763870,15691627.24,15734029.68,626570.64,584168.19,0,0,1,0,0,0,0,1,0
3,561392,22,22201.76,C2014200800,31748.82,9547.06,1553217.73,1575419.49,0,0,0,1,0,0,0,1,0
4,768127,39,129617.14,C973219994,16571.00,0.00,413656.24,543273.38,0,0,0,1,0,0,0,1,0


In [328]:
del newdf['nameOrig']

In [329]:
del newdf['index']

In [331]:
y=newdf['isFraud']
del newdf['isFraud']

In [332]:
newdf.head(2)

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,nameDest_C,nameDest_M
0,251,77799.25,0.0,0.0,845019.9,922819.15,0,0,1,0,0,0,1,0
1,155,9258.43,0.0,0.0,0.0,0.00,0,0,0,0,1,0,0,1


In [336]:
from sklearn.preprocessing import StandardScaler
# define standard scaler
scaler = StandardScaler()
# transform data
scaled = scaler.fit_transform(newdf)


In [339]:
newdf = pd.DataFrame(scaled)
newdf.columns = ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig','oldbalanceDest','newbalanceDest','isFlaggedFraud','type_CASH_IN','type_CASH_OUT','type_DEBIT','type_PAYMENT','type_TRANSFER','nameDest_C','nameDest_M']
newdf.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,nameDest_C,nameDest_M
0,0.051284,-0.155202,-0.287638,-0.291423,-0.072823,-0.079146,0.0,-0.526337,1.351612,-0.083456,-0.716009,-0.301807,0.716009,-0.716009
1,-0.623818,-0.258375,-0.287638,-0.291423,-0.303625,-0.309712,0.0,-0.526337,-0.739857,-0.083456,1.396631,-0.301807,-1.396631,1.396631
2,1.092065,-0.208484,5.228709,5.169242,-0.132489,-0.163758,0.0,1.899924,-0.739857,-0.083456,-0.716009,-0.301807,0.716009,-0.716009
3,-1.559115,-0.238891,-0.276477,-0.288110,0.120608,0.083906,0.0,-0.526337,1.351612,-0.083456,-0.716009,-0.301807,0.716009,-0.716009
4,-1.439565,-0.077201,-0.281813,-0.291423,-0.190642,-0.173975,0.0,-0.526337,1.351612,-0.083456,-0.716009,-0.301807,0.716009,-0.716009


In [340]:
y=pd.DataFrame(y)
y.head()

,isFraud
0,0
1,0
2,0
3,0
4,0


In [341]:
newdf.dtypes

step              float64
amount            float64
oldbalanceOrg     float64
newbalanceOrig    float64
oldbalanceDest    float64
newbalanceDest    float64
isFlaggedFraud    float64
type_CASH_IN      float64
type_CASH_OUT     float64
type_DEBIT        float64
type_PAYMENT      float64
type_TRANSFER     float64
nameDest_C        float64
nameDest_M        float64
dtype: object

In [343]:
newdf.shape, y.shape

((60000, 14), (60000, 1))

In [344]:
XTrain, XTest, YTrain, YTest = train_test_split(newdf, y, test_size = 0.20,stratify= y)

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the releas

In [345]:
XTrain.shape, XTest.shape, YTrain.shape, YTest.shape

((48000, 14), (12000, 14), (48000, 1), (12000, 1))

In [346]:
YTrain['isFraud'].value_counts()

0    47957
1       43
Name: isFraud, dtype: int64

In [347]:
YTest['isFraud'].value_counts()

0    11989
1       11
Name: isFraud, dtype: int64

In [349]:
import sklearn.metrics as metrics
from sklearn import svm
svclassifier = svm.SVC(kernel='poly',degree=3)
svclassifier.fit(XTrain, YTrain)
y_pred = svclassifier.predict(XTrain)
print(confusion_matrix(YTrain,y_pred))
y_pred1 = svclassifier.predict(XTest)
print(confusion_matrix(YTest,y_pred1))

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[47957     0]
 [   35     8]]
[[11989     0]
 [    9     2]]


In [376]:
def kfold(kernell,degreee,regPrameter):
    
    cv=KFold(n_splits=5)
    s=XTrain.to_numpy()
    yy=YTrain.to_numpy()
    if kernell=="poly":
        
        svclassifier = svm.SVC(kernel=kernell,degree=degreee, C=regPrameter)
    else:
        svclassifier = svm.SVC(kernel=kernell,C=regPrameter)
    for train_index, test_index in cv.split(s):
        X_train, y_train = s[train_index], yy[train_index]
        X_test, y_test = s[test_index], yy[test_index]
        svclassifier.fit(X_train, y_train)
        y_pred = svclassifier.predict(X_train)
        accur=accuracy_score(y_train,y_pred)
        print("Train Data Information ")
        print(confusion_matrix(y_train,y_pred))
        #print(classification_report(y_test,y_pred))
        print("CV Data Information ")
        y_pred1 = svclassifier.predict(X_test)
        accurr=accuracy_score(y_test,y_pred1)
        #print("Accuracy: ",accurr)
        print(confusion_matrix(y_test,y_pred1))
        print(classification_report(y_test,y_pred1))
        print("Test Data Information ")
        y_pred2 = svclassifier.predict(XTest)
        accurr=accuracy_score(YTest,y_pred2)
        print("Accuracy: ",accurr)
        print(confusion_matrix(YTest,y_pred2))
        print(classification_report(YTest,y_pred2))

In [377]:
i=1
parameter=[0.1,0.01,1,10.24]
while i<5:
    s=0
    while s<4:
        print("Polynomial Degree: "+str(i)+"RegParameter: "+str(parameter[s]))
        kfold('poly',i,parameter[s])
        s+=1
    i+=1

Polynomial Degree: 1RegParameter: 0.1


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for

Train Data Information 
[[38370     0]
 [   30     0]]
CV Data Information 
[[9587    0]
 [  13    0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9587
           1       0.00      0.00      0.00        13

    accuracy                           1.00      9600
   macro avg       0.50      0.50      0.50      9600
weighted avg       1.00      1.00      1.00      9600

Test Data Information 
Accuracy:  0.9990833333333333
[[11989     0]
 [   11     0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11989
           1       0.00      0.00      0.00        11

    accuracy                           1.00     12000
   macro avg       0.50      0.50      0.50     12000
weighted avg       1.00      1.00      1.00     12000



C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.boo

Train Data Information 
[[38364     0]
 [   36     0]]
CV Data Information 
[[9593    0]
 [   7    0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9593
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9600
   macro avg       0.50      0.50      0.50      9600
weighted avg       1.00      1.00      1.00      9600

Test Data Information 
Accuracy:  0.9990833333333333
[[11989     0]
 [   11     0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11989
           1       0.00      0.00      0.00        11

    accuracy                           1.00     12000
   macro avg       0.50      0.50      0.50     12000
weighted avg       1.00      1.00      1.00     12000



...

Train Data Information 
[[38364     0]
 [   25    11]]
CV Data Information 
[[9593    0]
 [   4    3]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9593
           1       1.00      0.43      0.60         7

    accuracy                           1.00      9600
   macro avg       1.00      0.71      0.80      9600
weighted avg       1.00      1.00      1.00      9600

Test Data Information 
Accuracy:  0.9993333333333333
[[11988     1]
 [    7     4]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11989
           1       0.80      0.36      0.50        11

    accuracy                           1.00     12000
   macro avg       0.90      0.68      0.75     12000
weighted avg       1.00      1.00      1.00     12000



C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto'

Train Data Information 
[[38370     0]
 [   20    10]]
CV Data Information 
[[9587    0]
 [  11    2]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9587
           1       1.00      0.15      0.27        13

    accuracy                           1.00      9600
   macro avg       1.00      0.58      0.63      9600
weighted avg       1.00      1.00      1.00      9600

Test Data Information 
Accuracy:  0.9993333333333333
[[11989     0]
 [    8     3]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11989
           1       1.00      0.27      0.43        11

    accuracy                           1.00     12000
   macro avg       1.00      0.64      0.71     12000
weighted avg       1.00      1.00      1.00     12000



C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto'

Train Data Information 
[[38360     0]
 [   27    13]]
CV Data Information 
[[9596    1]
 [   2    1]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9597
           1       0.50      0.33      0.40         3

    accuracy                           1.00      9600
   macro avg       0.75      0.67      0.70      9600
weighted avg       1.00      1.00      1.00      9600

Test Data Information 
Accuracy:  0.9994166666666666
[[11989     0]
 [    7     4]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11989
           1       1.00      0.36      0.53        11

    accuracy                           1.00     12000
   macro avg       1.00      0.68      0.77     12000
weighted avg       1.00      1.00      1.00     12000



In [379]:
i=1
parameter=[0.1,0.01,1,10.24]

s=0
while s<4:
    print("Sigmoid Kernel: "+"RegParameter: "+str(parameter[s]))
    kfold('sigmoid',i,parameter[s])
    s+=1
    

Sigmoid Kernel: RegParameter: 0.1
Train Data Information 


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for

[[38370     0]
 [   30     0]]
CV Data Information 
[[9587    0]
 [  13    0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9587
           1       0.00      0.00      0.00        13

    accuracy                           1.00      9600
   macro avg       0.50      0.50      0.50      9600
weighted avg       1.00      1.00      1.00      9600

Test Data Information 
Accuracy:  0.9990833333333333
[[11989     0]
 [   11     0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11989
           1       0.00      0.00      0.00        11

    accuracy                           1.00     12000
   macro avg       0.50      0.50      0.50     12000
weighted avg       1.00      1.00      1.00     12000



C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.boo

Train Data Information 
[[38364     0]
 [   36     0]]
CV Data Information 
[[9593    0]
 [   7    0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9593
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9600
   macro avg       0.50      0.50      0.50      9600
weighted avg       1.00      1.00      1.00      9600

Test Data Information 
Accuracy:  0.9990833333333333
[[11989     0]
 [   11     0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11989
           1       0.00      0.00      0.00        11

    accuracy                           1.00     12000
   macro avg       0.50      0.50      0.50     12000
weighted avg       1.00      1.00      1.00     12000



...

Train Data Information 
[[38336    28]
 [   36     0]]
CV Data Information 
[[9588    5]
 [   7    0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9593
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9600
   macro avg       0.50      0.50      0.50      9600
weighted avg       1.00      1.00      1.00      9600

Test Data Information 
Accuracy:  0.9981666666666666
[[11977    12]
 [   10     1]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11989
           1       0.08      0.09      0.08        11

    accuracy                           1.00     12000
   macro avg       0.54      0.54      0.54     12000
weighted avg       1.00      1.00      1.00     12000



C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto'

Train Data Information 
[[38349    21]
 [   30     0]]
CV Data Information 
[[9579    8]
 [  13    0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9587
           1       0.00      0.00      0.00        13

    accuracy                           1.00      9600
   macro avg       0.50      0.50      0.50      9600
weighted avg       1.00      1.00      1.00      9600

Test Data Information 
Accuracy:  0.9984166666666666
[[11980     9]
 [   10     1]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11989
           1       0.10      0.09      0.10        11

    accuracy                           1.00     12000
   macro avg       0.55      0.55      0.55     12000
weighted avg       1.00      1.00      1.00     12000



C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto'

Train Data Information 
[[38329    31]
 [   40     0]]
CV Data Information 
[[9592    5]
 [   3    0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9597
           1       0.00      0.00      0.00         3

    accuracy                           1.00      9600
   macro avg       0.50      0.50      0.50      9600
weighted avg       1.00      1.00      1.00      9600

Test Data Information 
Accuracy:  0.9980833333333333
[[11976    13]
 [   10     1]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11989
           1       0.07      0.09      0.08        11

    accuracy                           1.00     12000
   macro avg       0.54      0.54      0.54     12000
weighted avg       1.00      1.00      1.00     12000



In [380]:
i=1
parameter=[0.1,0.01,1,10.24]

s=0
while s<4:
    print("rbf Kernel: "+"RegParameter: "+str(parameter[s]))
    kfold('rbf',i,parameter[s])
    s+=1
    

rbf Kernel: RegParameter: 0.1


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for

Train Data Information 
[[38370     0]
 [   30     0]]
CV Data Information 
[[9587    0]
 [  13    0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9587
           1       0.00      0.00      0.00        13

    accuracy                           1.00      9600
   macro avg       0.50      0.50      0.50      9600
weighted avg       1.00      1.00      1.00      9600

Test Data Information 
Accuracy:  0.9990833333333333
[[11989     0]
 [   11     0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11989
           1       0.00      0.00      0.00        11

    accuracy                           1.00     12000
   macro avg       0.50      0.50      0.50     12000
weighted avg       1.00      1.00      1.00     12000



C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.boo

Train Data Information 
[[38364     0]
 [   36     0]]
CV Data Information 
[[9593    0]
 [   7    0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9593
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9600
   macro avg       0.50      0.50      0.50      9600
weighted avg       1.00      1.00      1.00      9600

Test Data Information 
Accuracy:  0.9990833333333333
[[11989     0]
 [   11     0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11989
           1       0.00      0.00      0.00        11

    accuracy                           1.00     12000
   macro avg       0.50      0.50      0.50     12000
weighted avg       1.00      1.00      1.00     12000



...

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto'

Train Data Information 
[[38370     0]
 [   22     8]]
CV Data Information 
[[9587    0]
 [  12    1]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9587
           1       1.00      0.08      0.14        13

    accuracy                           1.00      9600
   macro avg       1.00      0.54      0.57      9600
weighted avg       1.00      1.00      1.00      9600

Test Data Information 
Accuracy:  0.9991666666666666
[[11989     0]
 [   10     1]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11989
           1       1.00      0.09      0.17        11

    accuracy                           1.00     12000
   macro avg       1.00      0.55      0.58     12000
weighted avg       1.00      1.00      1.00     12000



C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto'

Train Data Information 
[[38360     0]
 [   29    11]]
CV Data Information 
[[9597    0]
 [   3    0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9597
           1       0.00      0.00      0.00         3

    accuracy                           1.00      9600
   macro avg       0.50      0.50      0.50      9600
weighted avg       1.00      1.00      1.00      9600

Test Data Information 
Accuracy:  0.9991666666666666
[[11989     0]
 [   10     1]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11989
           1       1.00      0.09      0.17        11

    accuracy                           1.00     12000
   macro avg       1.00      0.55      0.58     12000
weighted avg       1.00      1.00      1.00     12000



C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
